In [1]:
from fastlane_bot import Config, ConfigDB, ConfigNetwork, ConfigProvider
from fastlane_bot.bot import CarbonBot
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC, CPCContainer, T
from fastlane_bot.helpers import TxHelper
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonBot))
from fastlane_bot.testing import *
plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("2.0", __VERSION__)

Using default database url, if you want to use a different database, set the backend_url found at the bottom of manager_base.py
Error adding Ethereum blockchain to database Ethereum, (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "ix_blockchains_name"
DETAIL:  Key (name)=(Ethereum) already exists.

[SQL: INSERT INTO blockchains (name, block_number) VALUES (%(name)s, %(block_number)s) RETURNING blockchains.id]
[parameters: {'name': 'Ethereum', 'block_number': 0}]
(Background on this error at: https://sqlalche.me/e/20/gkpj) skipping...
Error adding exchange carbon_v1 to database, (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "exchanges_pkey"
DETAIL:  Key (id)=(6) already exists.

[SQL: INSERT INTO exchanges (id, name, blockchain_name) VALUES (%(id)s, %(name)s, %(blockchain_name)s)]
[parameters: {'id': 6, 'name': 'carbon_v1', 'blockchain_name': 'Ethereum'}]
(Background on this error at: https://sqlalche.me/e/20/gkpj) ski

C:\Users\nicho\AppData\Local\Temp\ipykernel_52372\1801012949.py:8: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-dark')


Version = 3-b2.1 [requirements >= 2.0 is met]


# Testing the _run functions on TENDERLY [NBTest022]

## MAINNET Configuration

### Set up the bot and curves

In [2]:
C = Config.new(config=Config.CONFIG_TENDERLY)
assert C.DATABASE == C.DATABASE_POSTGRES
# assert C.POSTGRES_DB == "defaultdb"
assert C.NETWORK == C.NETWORK_TENDERLY
assert C.PROVIDER == C.PROVIDER_TENDERLY
bot = CarbonBot(ConfigObj=C)
assert str(type(bot.db)) == "<class 'fastlane_bot.db.manager.DatabaseManager'>"

In [3]:
CCm = bot.get_curves()
exch = {c.P("exchange") for c in CCm}
print("Number of curves:", len(CCm))
print("Number of tokens:", len(CCm.tokens()))
print("Exchanges:", exch)

Number of curves: 3527
Number of tokens: 2230
Exchanges: {'carbon_v1', 'bancor_v2', 'sushiswap_v2', 'uniswap_v2', 'uniswap_v3', 'bancor_v3'}


In [4]:
used_curves = [
    '340282366920938463463374607431768211527-0', #actually deleted from carbon
    '340282366920938463463374607431768211527-1', #actually deleted from carbon
    '2722258935367507707706996859454145691730-1',
    '1701411834604692317316873037158841057360-1',
    '1701411834604692317316873037158841057360-0',
    '9187623906865338513511114400657741709390-0',
    '12590447576074723148144860474975423823947-0',
    '13951577043758477001998358904702496669779-1'
]
subcurves = [x for x in CCm if x.cid not in used_curves]
no_0x0 = [x for x in subcurves if '0x0-1AD5' not in x.pair]
CCm2 = CPCContainer([x for x in no_0x0 if x.params.exchange not in ['bancor_v2']])
len(CCm2)

3422

In [5]:
pairs = [x.pair.split('/') for x in CCm if x.params.exchange == 'carbon_v1']
len(set([x for itm in pairs for x in itm]))

27

In [6]:
pairs = [x.pair for x in CCm if x.params.exchange == 'carbon_v1']
len(set(pairs))

58

In [7]:
CCm.bycid("f1112f0481b940058d16a49759a9bb3a").p

28760.523861948794

In [8]:
CCm.bycid("5d5cebf33c0049258c40db6bc90b1558").p

28579.95448534522

In [9]:
# CCm.bycid("9a52c5bdef1c402497d1c4f5d7501f61").p

In [10]:
CCm.bycid("12590447576074723148144860474975423823947-0").p

30899.999999999673

In [11]:
1/CCm.bycid("12590447576074723148144860474975423823947-1").p

31000.001577363997

## Actual run

In [12]:
flt = [T.USDC, T.USDT, T.ETH, T.LINK, T.BNT, T.WBTC, T.DAI]
# bot._run(flashloan_tokens=flt, CCm=CCm2)

## Analytics

In [13]:
ops = bot._find_arbitrage_opportunities_carbon_single_pairwise(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_ARBOPPS)

c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1157: RuntimeWarning: invalid value encountered in subtract
  jac[:, j] = (func(x_plus) - y) / Dxj
c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastla

In [14]:
ops

(Decimal('3423.17134343467318019165433977510787513049914099205750517285196110606193542480'),
                                              WETH-6Cc2     UNI-F984
 e7dfee0364fb40f2b959ec0020f9a7ea             10.117086 -3666.666679
 13951577043758477001998358904702496669779-0 -11.000000  3666.666664
 AMMIn                                        10.117086  3666.666664
 AMMOut                                      -11.000000 -3666.666679
 TOTAL NET                                    -0.882914    -0.000015,
 [{'cid': 'e7dfee0364fb40f2b959ec0020f9a7ea',
   'tknin': 'WETH-6Cc2',
   'amtin': 10.117086148569797,
   'tknout': 'UNI-F984',
   'amtout': -3666.6666792128235,
   'error': None},
  {'cid': '13951577043758477001998358904702496669779-0',
   'tknin': 'UNI-F984',
   'amtin': 3666.666664123535,
   'tknout': 'WETH-6Cc2',
   'amtout': -11.0,
   'error': None}],
 'WETH-6Cc2',
 (CPCArbOptimizer.TradeInstruction(cid='e7dfee0364fb40f2b959ec0020f9a7ea', tknin='WETH-6Cc2', amtin=10.117086148569797,

In [15]:
# ord = bot._run(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_ORDSCAL)
# ord

In [16]:
# ti = bot._run(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_TI)
# ti

In [17]:
# ti

In [18]:
# ops = bot._find_arbitrage_opportunities(flashloan_tokens=flt, CCm=CCm, result=bot.AO_CANDIDATES)
# ops

In [19]:
# candf = pd.DataFrame(ops).sort_values(by=0, ascending=False)
# candf.reset_index(inplace=True, drop=True)
# candf.iloc[5][1]

In [20]:
# candf.iloc[0][0]

In [21]:
exact = bot._run(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_EXACT)
exact

c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1157: RuntimeWarning: invalid value encountered in subtract
  jac[:, j] = (func(x_plus) - y) / Dxj
c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastla

[_calc_carbon_output] before decode:  11000000000000000000 11000000000000000000 0 15417019411766
[_calc_carbon_output] after decode:  11 11 0.0 0.05477225575051392


[TradeInstruction(ConfigObj=Config(network=_ConfigNetworkTenderly(), db=_ConfigDBPostgres(), logger=_ConfigLoggerDefault(), provider=_ConfigProviderTenderly()), cid='e7dfee0364fb40f2b959ec0020f9a7ea', tknin='WETH-6Cc2', amtin=Decimal('10.106969062421227'), tknout='UNI-F984', amtout=Decimal('3652.019646265128648498'), pair_sorting='', raw_txs='[]', custom_data='', db=DatabaseManager(ConfigObj=Config(network=_ConfigNetworkTenderly(), db=_ConfigDBPostgres(), logger=_ConfigLoggerDefault(), provider=_ConfigProviderTenderly()), contracts={}, filters=[], data=                                       cid                last_updated  \
 0     00125d26-4f9d-4936-9a46-7e7708cee9b5  2023-05-06 19:04:10.202420   
 1219  5bbd01d2-ba90-49e8-8235-0f5183a44204  2023-05-06 19:11:11.844129   
 1231  5c976f61-5847-4219-a2ee-0b1eb96b2775  2023-05-06 19:18:59.176213   
 1230  5c87da80-15a0-4c59-baac-e8e9661ee920  2023-05-06 19:00:47.931782   
 2635  c21eb1b9-cf04-4ec5-bad1-77fb1b29b38b  2023-05-06 19:13:37.07

In [22]:
# agg = bot._run(flashloan_tokens=flt, CCm=CCm, result=bot.XS_AGGTI)
# agg

In [23]:
# agg[0].raw_txs

In [24]:
# agg[1].raw_txs

In [25]:
exact[0].amtin_wei, exact[0].amtout_wei

(10106969062421227000, 3652019646265128648498)

In [26]:
exact[1].amtin_wei, exact[1].amtout_wei

(3652019646265128648498, 10934146820916720630)

In [27]:
route_struct, flashloan_amount, flashloan_token_address = bot._run(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_ROUTE)

c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1157: RuntimeWarning: invalid value encountered in subtract
  jac[:, j] = (func(x_plus) - y) / Dxj
c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastla

[_calc_carbon_output] before decode:  11000000000000000000 11000000000000000000 0 15417019411766
[_calc_carbon_output] after decode:  11 11 0.0 0.05477225575051392
Error quantizing amount:  1.0106969062421227E-
WETH-6Cc2 [] True
trade_instruction.cid: e7dfee0364fb40f2b959ec0020f9a7ea
trade_instruction.cid: 13951577043758477001998358904702496669779


In [28]:
route_struct, flashloan_amount, flashloan_token_address

([{'exchangeId': 4,
   'targetToken': '0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984',
   'minTargetAmount': 1,
   'deadline': 1683689055,
   'customAddress': '0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984',
   'customInt': 3000,
   'customData': '0x'},
  {'exchangeId': 6,
   'targetToken': '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE',
   'minTargetAmount': 1,
   'deadline': 1683689055,
   'customAddress': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
   'customInt': 2000,
   'customData': '0x0000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000029000000000000000000000000000000530000000000000000000000000000000000000000000000c5f9f4270c7cb7ff32'}],
 10106969062421227000,
 '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE')

# Tenderly Tx Submission

In [29]:
# C.w3.provider.endpoint_uri

In [30]:
# C.w3.eth.getBlock(C.w3.eth.block_number).timestamp

In [31]:
# C.provider.CARBON_CONTROLLER_CONTRACT.strategy(13951577043758477001998358904702496669788)

In [32]:
### This to manually submit
bot._validate_and_submit_transaction_tenderly(
        ConfigObj = C,
        route_struct = route_struct,
        src_amount = flashloan_amount,
        src_address = flashloan_token_address,
            )

{'exchangeId': 4, 'targetToken': '0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984', 'minTargetAmount': 1, 'deadline': 1683689055, 'customAddress': '0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984', 'customInt': 3000, 'customData': '0x'}


{'exchangeId': 6, 'targetToken': '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE', 'minTargetAmount': 1, 'deadline': 1683689055, 'customAddress': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', 'customInt': 2000, 'customData': '0x0000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000029000000000000000000000000000000530000000000000000000000000000000000000000000000c5f9f4270c7cb7ff32'}


Submitting transaction to Tenderly...src_amount=10106969062421227000 src_address=0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE


AttributeDict({'transactionHash': HexBytes('0x102462f5f7db08db4d9d846d3cdda19506d10e7400ef19cde7674140fe3e52bc'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xeebfdcb031563f15a9f4560d590e04da93cb88ef718ecd6f19c876e546a2dfb4'),
 'blockNumber': 17213273,
 'from': '0x28C6c06298d514Db089934071355E5743bf21d60',
 'to': '0x41Eeba3355d7D6FF628B7982F3F9D055c39488cB',
 'cumulativeGasUsed': 488199,
 'gasUsed': 488199,
 'effectiveGasPrice': 0,
 'contractAddress': None,
 'logs': [AttributeDict({'logIndex': 0,
   'address': '0x649765821D9f64198c905eC0B2B037a4a52Bc373',
   'topics': [HexBytes('0xc322efa58c9cb2c39cfffdac61d35c8643f5cbf13c6a7d0034de2cf18923aff3'),
    HexBytes('0x000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'),
    HexBytes('0x000000000000000000000000eef417e1d5cc832e619ae18d2f140de2999dd4fb'),
    HexBytes('0x00000000000000000000000041eeba3355d7d6ff628b7982f3f9d055c39488cb')],
   'data': '0x0000000000000000000000000000000000000000000000008c432ad0317ae5f8',
   'b

# MAINNNET Preferred Tx Submission Method

In [33]:
# from fastlane_bot.helpers import TxHelpers

In [34]:
# pool = (
#     bot.db.get_pool(exchange_name=bot.ConfigObj.BANCOR_V3_NAME, pair_name='BNT-FF1C/ETH-EEeE')
# )
# bnt_eth = (int(pool.tkn0_balance), int(pool.tkn1_balance))

# # Init TxHelpers
# tx_helpers = TxHelpers(ConfigObj=bot.ConfigObj)
# # Submit tx



In [35]:
# tx_helpers.validate_and_submit_transaction(route_struct=route_struct, src_amt=flashloan_amount,
#                                                     src_address=flashloan_token_address, bnt_eth=bnt_eth,
#                                                     expected_profit=Decimal('1000'), safety_override=True)

# MAINNET Forced Tx Submission Method

In [36]:
# # Initialize tx helper
# tx_helper = TxHelper(
#     usd_gas_limit=55,
#     ConfigObj=bot.ConfigObj,
#     gas_price_multiplier = 1
# )
# # # Submit the transaction
# # tx_helper.submit_flashloan_arb_tx(
# #     arb_data=route_struct,
# #     flashloan_token_address=flashloan_token_address,
# #     flashloan_amount=flashloan_amount,
# #     verbose=True,
# # )

In [37]:
# tx_helper.gas_limit